# SAE 2.02 - Exploration algorithmique d'un problème
## Ludmann Dorian et Maupou Cassandra  
### Question 1:


In [17]:
import networkx as nx
import json

def charger_resultats(nom_fichier): #O(N²)
    """charge un fichier de résultats au DNB donné au format CSV en une liste de résultats

    Args:
        nom_fichier (str): nom du fichier CSV contenant les résultats au DNB

    Returns:
        list: la liste des résultats contenus dans le fichier
    """
    #Initialisation
    liste_resultats = {}
    fic = open(nom_fichier, 'r')
    fic.readline()
    #Pour chaque ligne du fichier
    for ligne in fic:
        titrePresent = False
        dic_ligne = json.loads(ligne)
        dic_translated_ligne = {}
        #Pour chaque clé et valeur associé au dictionnaire de la ligne
        for(key,value) in dic_ligne.items():
            if key == "title":
                title_value = value
                titrePresent = True
            elif key != "year":
                value2 = []
                for elem in value:
                    elem2 = elem.replace("]]","").replace("[[","")
                    value2.append(elem2)
                dic_translated_ligne[key] = value2
            else:
                dic_translated_ligne[key] = value
        #Ajout au dictionnaire si et seulement si un titre a été identifier
        if titrePresent:
            liste_resultats[title_value] = dic_translated_ligne
    return liste_resultats

def getGraph(dic ,dir_Included=False): #O(N³)
    """Construit un graphe networkx à partir d'un dictionnaire
    Args:
        dic (dic): Le dictionnaire des données du graphe
        dir_Included (boolean): Booléen auquel est associé si les directeurs deviennent sommet dans le graphe ou non
    """
    G = nx.Graph()
    i = 0
    bool = False
    for value in dic.values():
        if dir_Included:
            ensemble = set()
            for(key,item) in value.items():
                if key == "cast" or key == "directors":
                    for personne in item:
                        ensemble.add(personne)    
            for elem1 in ensemble:
                for elem2 in ensemble:
                    if elem1 != elem2:
                        G.add_edge(elem1,elem2)
        else:
            for elem1 in value["cast"]:
                for elem2 in value["cast"]:
                    if elem1 != elem2:
                        G.add_edge(elem1,elem2)
        
    return(G)

In [18]:
dic_Bacon = charger_resultats("smaller_data.txt")
G = getGraph(dic_Bacon)

### Question 2:

In [19]:
def collaborateurCommun(G, personne1, personne2): #O(N)
    """Donne la liste des acteurs ayant pour collaborateur direct deux acteurs donnés

    Args:
        G (networkx.graph): Le graphe
        personne1 (str): le nom de l'acteur 1
        personne2 (str): le nom de l'acteur 2
    Returns:
        list: la liste des acteurs étant collaborateur commun des deux acteurs donnés en paramètre
    """
    res = []
    for collab in G[personne1]:
        if collab in G[personne2] and collab != personne1:
            res.append(collab)
    return res

In [20]:
print(collaborateurCommun(G, "Joe Turkel", "Harrison Ford"))

['Rutger Hauer', 'Sean Young', 'Edward James Olmos', 'M. Emmet Walsh', 'Daryl Hannah', 'William Sanderson', 'Brion James', 'Joanna Cassidy', 'James Hong', 'Morgan Paull', 'Hy Pyke']


#### a. Comment exprimeriez-vous cette notion (ensemble de collaborateurs en commun) en termes de théorie des graphes ?  
En théorie des graphes, l'ensemble des collaborateurs communs entre deux acteurs a1 et a2 est l'ensemble des sommets qui ont au minimum pour voisin les deux acteurs a1 et a2.

#### b. Pouvez-vous donner une borne inférieure sur le temps nécessaire à l'exécution de votre fonction ?  
O(1) <= T<sub>éxécution</sub> <= N² avec N = L'ordre du graphe G

### Question 3:

#### a. Reconnaissez-vous l'algorithme classique en théorie des graphes qui est au coeur de ce programme ?
Cet algorithme reprend le BFS (Breadth-First Search, ou parcours en largeur) pour parcourir jusqu'à une distance donné l'ensemble des acteurs lié par une distance inférieure ou égal à l'acteur initiale.

#### b. Grâce à la fonction précédente, comment pouvez-vous déterminer si un acteur se trouve à distance k d'un autre acteur ?
Il suffit de modifier le code de cette facon suivante!

In [21]:
def acteursDistanceDonnee(G,u,e, k): #O(N³)
    """Fonction renvoyant si l'acteur u et e sont séparé d'une distance k
    
    Parametres:
        G (networkx.graph): Le graphe
        u (str): le sommet de départ
        e (str): le sommet d'arrivée
        k (int): la distance depuis u
    """
    if u not in G.nodes:
        print(u,"est un illustre inconnu")
        return None
    collaborateurs = set()
    collaborateurs.add(u)
    for i in range(k):
        collaborateurs_directs = set()
        for c in collaborateurs:
            for voisin in G.adj[c]:
                if voisin not in collaborateurs:
                    collaborateurs_directs.add(voisin)
                    if voisin == e and i == k:
                        return True
        collaborateurs = collaborateurs.union(collaborateurs_directs)
    return False

#### c. Tentons maintenant de déterminer la distance entre deux acteurs. Est-ce que réutiliser la fonction précédente vous semble intéressant ? Donnez la complexité d'un tel algorithme.
Il peut être intéressant de reprendre la fonction donnée, de part le fait que la fonction utilise aussi le BFS.

#### d. Comment pouvez-vous maintenant modifier la fonction qui vous a été fournie afin de trouver la distance entre deux acteurs ?

In [32]:
def distance_acteur(G, act1, act2): #O(ΣN + ΣE)
    """Donne la distance entre deux acteurs donnés

    Args:
        G (networkx.graph): Le graphe
        act1 (str): le nom de l'acteur 1
        act2 (str): le nom de l'acteur 2
    Returns:
        int: la distance entre les acteurs
    """
    if act1 not in G.nodes or act2 not in G.nodes:
        return None
    if act1 == act2:
        return 0
    if act1 in G.adj[act2]:
        return 1
    
    collaborateurs_A = set()
    collaborateurs_A.add(act1)
    collaborateurs_B = set()
    collaborateurs_B.add(act2)
    acteur_commun = False
    distance = 1

    while not acteur_commun:
        # Parcours depuis first_acteur
        collaborateurs_directs_A = set()
        for acteur in collaborateurs_A:
            for voisin in G.adj[acteur]:
                if voisin not in collaborateurs_A:
                    collaborateurs_directs_A.add(voisin)
        
        # Parcours de puis second_acteur
        collaborateurs_directs_B = set()
        for acteur in collaborateurs_B:
            for voisin in G.adj[acteur]:
                if voisin not in collaborateurs_B:
                    # Tests si les deux parcours se rejoignent
                    if voisin in collaborateurs_A:
                        return distance * 2
                    if voisin in collaborateurs_directs_A:
                        return distance * 2 - 1
                    collaborateurs_directs_B.add(voisin)

        distance += 1
        collaborateurs_A = collaborateurs_A.union(collaborateurs_directs_A)
        collaborateurs_B = collaborateurs_B.union(collaborateurs_directs_B)

In [33]:
print(distance_acteur(G, 'Michael Caine','James Stewart'))
print(distance_acteur(G, 'Leon Vitali','Michael Caine'))

1
4


#### e. Donnez la complexité d'un tel algorithme.
Cette fonction possède alors une compléxité de O(ΣN + ΣE) avec N sommet appartenant au graphe et E les arrêtes du graphe. La complexité de cet algorithme est donc égal à la somme du nombre de sommet et d'arrêtes d'un graphe G.

### Question 4:
On cherche maintenant à déterminer la centralité d'un acteur. Pour cela, on veut déterminer la plus grande distance qui le sépare d'un autre acteur dans le graphe.

#### a. Quelle notion de théorie des graphes permet de modéliser cela ? Proposez une fonction qui calcule la centralité d'un acteur dans Gc. Le parcours en profondeur pourrait aider à modéliser la plus grande distance entre deux acteurs dans un graphe.
La notion de théorie des graphes qui permet de modéliser cela est l'intermédiarité

In [45]:
def centralite_acteur(graph, acteur): #O(N²(logN))
    """Donne la centralité d'un acteur donné

    Args:
        graph (networkx.graph): Le graphe
        acteur (str): le nom de l'acteur
    Returns:
        int: la centralité de l'acteur
    """
    if acteur in graph.nodes():
        visites = dict()
        acteurs = []
        acteurs.append((acteur, 0))
        i = 0
        while len(acteurs) != 0:
            (acteur_courant, distance) = acteurs.pop(0)
            if acteur_courant not in visites.keys():
                visites[acteur_courant] = distance
                if distance > i:
                    i = distance
                for voisin in graph.adj[acteur_courant]:
                    if voisin not in visites.keys():
                        acteurs.append((voisin, distance + 1))
        return i
    
    else:
        return None

In [47]:
print(centralite_acteur(G,"Dick Jones (actor)|Dickie Jones"))
print(centralite_acteur(G,"Ludmann Dorian"))

5
None


#### b. A l'aide de la fonction précédente, écrivez une autre fonction qui va déterminer l'acteur le plus central du graphe Gc.

In [55]:
def centralite(graph): #O(N³(logN))
    """Donne l'acteur central du graphe

    Args:
        graph (networkx.graph): Le graphe
    Returns:
        str: l'acteur central du graphe
    """
    ppc = None
    acteur_res = ""
    centralite_a = 0
    for acteur in G.nodes():
        centralite_a = centralite_acteur(graph, acteur)
        if ppc == None or centralite_a < ppc:
            ppc = centralite_a
            acteur_res = acteur
    return acteur_res

In [56]:
print(centralite(G))

Bruce Campbell


### Question 5:
Nous allons maintenant tenter de déterminer si deux acteurs peuvent être très éloignés dans Gc. Plus précisément, vous fournirez une fonction permettant de déterminer la distance maximum dans Gc entre toute paire d'acteurs/actrices.

In [57]:
def plusGrandeDistanceEntreActeurs(graph): #O(N²*(ΣN + ΣE))
    """Retourne la plus grande distance possible entre toute paire d'acteur
    Args:
        graph (networkx.graph): Le graphe
    Returns:
        int: la plus grande distance possible
    """
    best_distance = None
    for acteur1 in G.nodes():
        for acteur2 in G.nodes():
            if acteur1 != acteur2:
                distance = distance_acteur(G, acteur1, acteur2)
                if best_distance == None or distance > best_distance:
                    best_distance = distance
    return best_distance

In [59]:
print(plusGrandeDistanceEntreActeurs(G))
#Même avec un jeu de donnée tout petit, la méthode possède un temps d'éxécution trop élevé pour pouvoir être utilisé

KeyboardInterrupt: 

### Question 6:

#### a. Proposez une méthode similaire à celle calculant le centre du graphe mais qui se restrint ici à déterminer le centre d'un groupe d'acteur (On remarquera qu'un tel sommet ne fait nécessairement partie du groupe en question).

In [60]:
def centre_groupe_acteurs(G, groupe_acteurs): #O(N)
    """
    Détermine le ou les acteurs central/centraux d'un groupe d'acteurs dans un graphe.

    Args:
        G (networkx.Graph): Le graphe.
        groupe_acteurs (list): Une liste d'acteurs.

    Returns:
        list(str): La liste des noms des acteurs centraux
    """
    res = []
    distance_min = None

    for acteur in groupe_acteurs:
        centralite = centralite_acteur(G, acteur)
        if distance_min is None or centralite < distance_min:
            distance_min = centralite
            res = [acteur]
        elif centralite == distance_min:
            res.append(acteur)

    return res

#### b. Faites en sorte, que quand cela a du sens, que vos fonctions renvoient un sous-graphe de Gc plutôt qu'une simple liste d'acteurs. Par exemple, proposer une vatiante de la fonction déterminant les collaborateurs proches d'un sommet v qui renverra le sous-graphe induit par v et par tous les sommets à distance k de v.

In [61]:
def sous_graphe_collaborateurs_communs(G, first_acteur, second_acteur): #O(N)
    """
    Renvoie le sous-graphe induit par les collaborateurs communs entre deux acteurs.

    Args:
        G (networkx.Graph): Le graphe.
        first_acteur (str): Le nom du premier acteur.
        second_acteur (str): Le nom du deuxième acteur.

    Returns:
        networkx.Graph: Le sous-graphe induit par les collaborateurs communs des acteurs de départ.
    """
    if first_acteur not in G.nodes or second_acteur not in G.nodes:
            return None
    if first_acteur == second_acteur:
        return G.subgraph([first_acteur])
    res = set(first_acteur, second_acteur)
    for acteur in G.adj[first_acteur]:
        if acteur in G.adj[second_acteur]:
            res.add(acteur)
    return G.subgraph(res)

### Question 7:
L'efficacité de l'ensemble de vos méthodes, tant sur le temps d'exécution que sur la mémoire utilisée, est bien entendu essentielle. Par exemple, calculer la distance entre 2 sommets peut se faire de différentes manières, comme évoqué précédemment, impliquant des temps de calcul différents. Il est égaliement possible de pré-calculer un certain nombre de distance et d'utiliser ces informations pour calculer des distances entre des paires de sommets plus éloignés. Pour cette dernière partie du travail, vous vous attacherez donc à proposer différentes implémentations du calcul de distance et à évaluer ces différentes propositions. La description de vos évaluations expérimentales ainsi que vos conclusions devront apparaître dans votre rapport.  

In [62]:
def distance_acteur(G, act1, act2, dic=None): #O(N²(logN))
    """Donne la distance entre deux acteurs donnés

    Args:
        G (networkx.graph): Le graphe
        act1 (str): le nom de l'acteur 1
        act2 (str): le nom de l'acteur 2
        dic (dic): le dictionnaire (K,V) avec K le couple de sommet, et V la distance entre ces deux sommets
    Returns:
        int: la distance entre les acteurs
    """
    if dic == None:
        dic = {}
    if act1 in G.nodes() and act2 in G.nodes():
        visited = []
        queue = [act1]
        i = -1
        while queue:
            node = queue.pop(0)
            if node == act2:
                        return i
            if (act1, node) in dic.keys():
                return dic[(act1, node)] + i
            if (node, act1) in dic.keys():
                return dic[(node, act1)] + i
            if node not in visited:
                dic[(act1, node)] = i
                visited.append(node)
                i += 1         
                for edge in G.edges:
                    if edge[0] == node:
                        queue.append(edge[1])
                    elif edge[1] == node:
                        queue.append(edge[0])
    return None